In [1]:
import os
import sys
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors

In [2]:
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "Data")))

In [3]:
import LoadData as LD

In [4]:
df = LD.load_data("../../Data/Raw/IMDB Dataset sample.csv")

In [5]:
import spacy

# Load the large language model
nlp = spacy.load("en_core_web_lg")

In [6]:
def preprocess_text(text):
    """
    Preprocess the input text using spaCy.

    Args:
    text (str): The text to preprocess.

    Returns:
    List[str]: A list of cleaned and lemmatized tokens.
    """
    # Process the text with spaCy
    doc = nlp(text)
    
    # Initialize an empty list to hold the cleaned tokens
    cleaned_tokens = []
    
    for token in doc:
        # Filter out stop words, punctuation, and tokens with less than 2 characters, spaces, numbers
        if not token.is_stop and not token.is_punct and len(token.text) > 1 and not token.is_space and not token.like_num:
            # Lemmatize the token and add to the cleaned tokens list
            cleaned_tokens.append(token.lemma_)
    
    return cleaned_tokens

In [7]:
df['review'] = df['review'].apply(lambda x: re.sub(r'<br /><br />', '', x))
df['review'] = df['review'].apply(lambda x: re.sub(r'\'', '', x))
df['cleaned_review'] = df['review'].apply(lambda x: preprocess_text(x))

KeyboardInterrupt: 

In [ ]:
df.to_excel("../../Data/Process/sample_preprocessed_data.xlsx", index=False)

# Vectorization

In [ ]:
def word_embedding(df, model, review_column='review'):

    # Tokenization function
    def preprocess(text):
        return re.findall(r'\b\w+\b', text.lower())
    
    # Function to get vector representation of a review
    def get_review_vector(model, tokens):
        # Get vectors for each word in the review
        word_vectors = [model.wv[word] for word in tokens if word in model.wv]
        # Calculate the mean of the vectors
        if len(word_vectors) == 0:  # Handle empty word_vectors case
            return np.zeros(model.vector_size)
        return np.mean(word_vectors, axis=0)

    # Apply tokenization to each review
    df['tokens'] = df[review_column].apply(preprocess)
    
    
    if model="word2vec":
        # Train Word2Vec model
        word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

        # Apply the function to each review for Word2Vec
        df['vector'] = df['tokens'].apply(lambda tokens: get_review_vector(word2vec_model, tokens))
        
    if model="fasttext":

        # Train FastText model
        fasttext_model = FastText(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

        # Apply the function to each review for FastText
        df['vector'] = df['tokens'].apply(lambda tokens: get_review_vector(fasttext_model, tokens))

    return df[['review', 'vector', 'sentiment']]

In [ ]:
def pre_trained_word_embedding(df, model, review_column='review'):

    # Tokenization function
    def preprocess(text):
        return re.findall(r'\b\w+\b', text.lower())
    
    # Function to get vector representation of a review
    def get_review_vector(model, tokens):
        # Get vectors for each word in the review
        word_vectors = [model[word] for word in tokens if word in model]
        # Calculate the mean of the vectors
        if len(word_vectors) == 0:  # Handle empty word_vectors case
            return np.zeros(model.vector_size)
        return np.mean(word_vectors, axis=0)

    # Apply tokenization to each review
    df['tokens'] = df[review_column].apply(preprocess)
    
    
    if model="word2vec":
        # Train Word2Vec model
        word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

        # Apply the function to each review for Word2Vec
        df['vector'] = df['tokens'].apply(lambda tokens: get_review_vector(word2vec_model, tokens))
        
    if model="fasttext":

        # Train FastText model
        fasttext_model = KeyedVectors.load('cc.en.300.bin')

        # Apply the function to each review for FastText
        df['vector'] = df['tokens'].apply(lambda tokens: get_review_vector(fasttext_model, tokens))

    return df[['review', 'vector', 'sentiment']]

In [ ]:
def vectorization(df, review_column='cleaned_review', vectorizer, pretrained=False):

    if vectorizer='tf-idf':
        # Create TfidfVectorizer instance
        vectorizer = TfidfVectorizer()
    elif vectorizer='count':
        # Create CountVectorizer instance
        vectorizer = CountVectorizer()
    else:
        if pretrained:
            return pre_trained_word_embedding(df, model=vectorizer, review_column='review')
        else:
            return word_embedding(df, model=vectorizer, review_column='review')
    

    # Fit and transform the data
    X = vectorizer.fit_transform(df[review_column])

    # Convert sparse matrix to dense format
    dense_matrix = X.toarray()

    # Get feature names (i.e., words)
    feature_names = vectorizer.get_feature_names_out()

    # Create a DataFrame for better readability
    df_counts = pd.DataFrame(dense_matrix, columns=feature_names)
    df['vector'] = df_counts.values.tolist()
    return df[['review', 'vector', 'sentiment']]